In [10]:
import numpy as np

In [2]:
dtrajs = np.load('/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo_FINAL/PAPER_FIGURES/bayesian_models/sam/67.npy')

In [3]:
dtrajs_glob = np.load('/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo_FINAL/PAPER_FIGURES/bayesian_models/sam/dtrajs_glob.npy')

In [4]:
dtrajs_glob[0]

'data_cut_start_proteinonly_noH/11710/run0-clone261.h5'

In [21]:
obs_prob = np.load('/Users/rafalpwiewiora/repos/MSM/11708_11710_SET8_ligands/dih/SAM/hmms/results/12/observation_probabilities.npy')

In [6]:
numbers_of_frames = np.zeros(100)

for x in dtrajs:
    for y in x:
        numbers_of_frames[y] += 1

In [10]:
np.sum(numbers_of_frames)

1955965.0

In [12]:
# add 0 obs_prob for the microstates not present

In [14]:
obs_prob.shape

(10, 67)

In [15]:
numbers_of_frames

array([  3.00000000e+00,   5.63000000e+02,   1.11000000e+02,
         1.89700000e+03,   3.01100000e+03,   0.00000000e+00,
         7.97000000e+03,   1.30700000e+04,   5.76600000e+03,
         0.00000000e+00,   0.00000000e+00,   5.22310000e+04,
         0.00000000e+00,   8.60000000e+01,   6.40000000e+01,
         5.34000000e+02,   1.04600000e+03,   0.00000000e+00,
         1.22400000e+03,   6.46600000e+03,   0.00000000e+00,
         6.49000000e+02,   0.00000000e+00,   1.11376000e+05,
         0.00000000e+00,   2.84970000e+04,   4.38400000e+03,
         4.15800000e+03,   0.00000000e+00,   9.14840000e+04,
         3.37000000e+02,   0.00000000e+00,   0.00000000e+00,
         2.21000000e+02,   0.00000000e+00,   0.00000000e+00,
         1.18439000e+05,   7.19000000e+02,   1.42730000e+04,
         0.00000000e+00,   0.00000000e+00,   9.87810000e+04,
         1.00000000e+00,   6.04000000e+02,   2.49491000e+05,
         1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,

In [22]:
obs_prob_ = []

j = 0
for i in range(100):
    if not numbers_of_frames[i] == 0:
        obs_prob_.append(obs_prob[:,j])
        j += 1
    else:
        obs_prob_.append([0,0,0,0,0,0,0,0,0,0])

In [23]:
np.array(obs_prob_).shape

(100, 10)

In [31]:
obs_prob = np.array(obs_prob_).T

In [32]:
weights = []

for i in range(10):
    weights_ = []
    for x in dtrajs:
        for y in x:
            weights_.append(obs_prob[i,y] / numbers_of_frames[y])
    weights.append(weights_)

In [33]:
np.sum(np.concatenate(weights))

10.000000000000009

In [34]:
traj_frame_indexes = []

for i,x in enumerate(dtrajs):
    for j in np.arange(len(x)):
        traj_frame_indexes.append((i, j))

In [35]:
draws = []

for i in range(10):
    draws_ = np.random.choice(np.arange(len(traj_frame_indexes)), size=100, replace=False, p=weights[i])
    draws__ = []
    for x in draws_:
        draws__.append(traj_frame_indexes[x])
    draws.append(draws__)    

In [36]:
draws_names = []

for x in draws:
    draws_names_ = []
    for y in x:
        draws_names_.append((dtrajs_glob[y[0]], y[1]))
    draws_names.append(draws_names_)    

In [37]:
len(draws_names)

10

In [38]:
np.save('sam_draws', draws_names)

In [39]:
del weights

In [40]:
del draws

In [42]:
draws_names = np.load('sam_draws.npy')

In [1]:
# collected frames on lilac - are in sam_samples

In [2]:
import mdtraj as md

In [39]:
samples = md.load('sam_samples/%d.pdb' % 0)

In [40]:
samples

<mdtraj.Trajectory with 100 frames, 1296 atoms, 162 residues, and unitcells at 0x61c442630>

In [41]:
samples_CA = samples.atom_slice(samples.top.select('name CA'))

In [42]:
samples_CA

<mdtraj.Trajectory with 100 frames, 162 atoms, 162 residues, and unitcells at 0x613e12a90>

In [43]:
rmsd_matrix_CA = np.zeros((100,100))

In [44]:
# SET
selection1 = np.concatenate([np.arange(25,59), np.arange(95,145)])

# SET-I + C-flank
selection2 = np.concatenate([np.arange(59,95), np.arange(145,162)])

In [45]:
for i in range(100):
    for j in range(100):
        trajectory = samples_CA[i]
        reference = samples_CA[j]                   
        trajectory.superpose(reference, atom_indices=selection1)
        rmsd_matrix_CA[i,j] = np.sqrt(3*np.mean((trajectory.xyz[:, selection2, :] - reference.xyz[:, selection2, :])**2, axis=(1,2)))

In [51]:
np.min(np.sum(rmsd_matrix_CA, axis=0) / 99)

0.39990791934307174

In [30]:
# ok do this for all macrostates now

In [52]:
rmsds = []

for k in range(10):
    samples = md.load('sam_samples/%d.pdb' % k)
    samples_CA = samples.atom_slice(samples.top.select('name CA'))
    rmsd_matrix_CA = np.zeros((100,100))
    for i in range(100):
        for j in range(100):
            trajectory = samples_CA[i]
            reference = samples_CA[j]                   
            trajectory.superpose(reference, atom_indices=selection1)
            rmsd_matrix_CA[i,j] = np.sqrt(3*np.mean((trajectory.xyz[:, selection2, :] - reference.xyz[:, selection2, :])**2, axis=(1,2)))
    rmsds.append(np.min(np.sum(rmsd_matrix_CA, axis=0) / 99))
    print(k)

0
1
2
3
4
5
6
7
8
9


In [56]:
rmsds

[0.39990791934307174,
 0.31007706011697594,
 0.41642058516727004,
 0.4526480201061186,
 0.27316211988853939,
 0.41835448793088548,
 0.40980302771056903,
 0.29559354526732212,
 0.30317665564896507,
 0.38944238711446]

In [57]:
np.save('sam_rmsds', rmsds)

In [58]:
np.savetxt('sam_rmsds.csv', rmsds, delimiter=",")

In [59]:
meta_sets = np.load('/Users/rafalpwiewiora/repos/MSM/11708_11710_SET8_ligands/dih/SAM/hmms/results/12/metastable_sets.npy')

In [60]:
meta_sets

array([array([ 2,  5,  7, 11, 21, 23, 25, 34, 54, 55, 60, 65]),
       array([13, 19]), array([ 0, 18, 31]), array([46]),
       array([ 1,  6, 28, 43, 48]),
       array([ 4, 16, 20, 30, 32, 33, 35, 39, 50, 58, 59, 62, 64]),
       array([44, 49, 51]),
       array([ 3,  9, 12, 15, 17, 22, 24, 27, 36, 37, 40, 42, 45, 47, 53, 56, 61,
       63, 66]),
       array([26, 29]), array([ 8, 10, 14, 38, 41, 52, 57])], dtype=object)

In [61]:
import glob

In [62]:
pdb_glob = glob.glob('/Users/rafalpwiewiora/repos/MSM/11708_11710_SET8_ligands/dih/SAM/cluster_centers_top_10/pdb/*.pdb')

In [63]:
pdb_glob_nos = [int(x.split('/')[-1][:-4]) for x in pdb_glob]
pdb_glob_nos = sorted(pdb_glob_nos)

In [84]:
rmsds = []

for k in range(10):
    meta_set = meta_sets[k]
    samples_ = []
    for micro in meta_set:
        micro = pdb_glob_nos[micro]
        samples_.append(md.load('/Users/rafalpwiewiora/repos/MSM/11708_11710_SET8_ligands/dih/SAM/cluster_centers_top_10/pdb/%d.pdb' % micro))
    samples = samples_[0]
    samples = samples.atom_slice(samples.top.select('protein and name CA'))
    for sample in samples_[1:]:
        sample = sample.atom_slice(sample.top.select('protein and name CA'))
        samples = md.join([samples, sample])
    samples_CA = samples.atom_slice(samples.top.select('name CA'))
    rmsd_matrix_CA = np.zeros((len(samples),len(samples)))
    for i in range(len(samples)):
        for j in range(len(samples)):
            trajectory = samples_CA[i]
            reference = samples_CA[j]                   
            trajectory.superpose(reference, atom_indices=selection1)
            rmsd_matrix_CA[i,j] = np.sqrt(3*np.mean((trajectory.xyz[:, selection2, :] - reference.xyz[:, selection2, :])**2, axis=(1,2)))
    rmsds.append(np.min(np.sum(rmsd_matrix_CA, axis=0) / (len(rmsd_matrix_CA)-1)))
    print(k)

0
1
2
3
4
5
6
7
8
9


In [85]:
rmsds

[0.42394448130154144,
 0.1524939560576489,
 0.34529050182061366,
 0.27376677080554285,
 0.32708862822521545,
 0.389373490971685,
 0.32477873157780973,
 0.31874776105913849,
 0.21900918773827149,
 0.27966884760260163]

In [86]:
np.save('sam_rmsds_top10', rmsds)

In [87]:
np.savetxt('sam_rmsds_top10.csv', rmsds, delimiter=",")